# Rare event testing

Identifies what features are relatively rare; i.e., whether it's a zero-inflated column.
This is just for insight, and isn't used for anything.

## Data loading

In [10]:
import pandas as pd
import python_proj.utils.exp_utils as exp_utils

input_data_file_name = "dataset_transformed"

data_path_format = exp_utils.BASE_PATH + "/final_data/{data_file_name}.csv"
data_path = data_path_format.format(data_file_name=input_data_file_name)
print(f"{data_path=}")

meta_header_count = 5
pr_merged_key = "PullRequestIsMerged"
first_time_contributor_key = "SubmitterIsFirstTimeContributor"

df: pd.DataFrame = pd.read_csv(filepath_or_buffer=data_path, header=0)

print(f"{len(df)=}.")
print(f"{len(df.columns)=}\n")

df.describe()

data_path='/workspaces/msc_thesis/data//final_data/dataset_transformed.csv'
len(df)=1216221.
len(df.columns)=51



,ID,Submitter ID,PR Number,IntraProjectSubmitterPullRequestSuccessRate,EcosystemExperienceSubmitterPullRequestSuccessRate,DependencyEcosystemExperienceSubmitterPullRequestSuccessRate,NonDependencyEcosystemExperienceSubmitterPullRequestSuccessRate,InversedDependencyEcosystemExperienceSubmitterPullRequestSuccessRate,ln(1 + ControlPullRequestLifeTimeInMinutes),ln(1 + ControlNumberOfCommitsInPullRequest),...,ln(1 + InversedDependencyEcosystemExperienceSubmitterIssueSubmissionCount),ln(1 + InversedDependencyEcosystemExperienceSubmitterIssueCommentCount),ln(1 + WeightedEcosystemSecondOrderInDegreeCentrality),ln(1 + WeightedEcosystemSecondOrderOutDegreeCentrality),ln(1 + WeightedIntraProjectSecondOrderInDegreeCentrality),ln(1 + WeightedIntraProjectSecondOrderOutDegreeCentrality),ln(1 + EcosystemIntegratorToSubmitterLinkIntensity),ln(1 + EcosystemSubmitterToIntegratorLinkIntensity),ln(1 + IntraProjectIntegratorToSubmitterLinkIntensity),ln(1 + IntraProjectSubmitterToIntegratorLinkIntensity)
count,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,...,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06,1.216221e+06
mean,1.473143e+08,5.093466e+06,8.082764e+02,5.027950e-01,4.028468e-01,9.859457e-02,3.751558e-01,5.921727e-02,4.195870e-01,1.184307e-01,...,9.537892e-03,1.753143e-02,5.184194e-02,4.184437e-02,4.874358e-02,4.224923e-02,4.721868e-02,5.028227e-02,5.627708e-02,7.173808e-02
std,9.953631e+07,7.967840e+06,2.296849e+03,4.710790e-01,4.523912e-01,2.907958e-01,4.479800e-01,2.284907e-01,2.256942e-01,7.152478e-02,...,5.259800e-02,8.246395e-02,8.803506e-02,8.230781e-02,9.031063e-02,8.361355e-02,1.028680e-01,1.043392e-01,1.113767e-01,1.174610e-01
min,6.870000e+02,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.925868e+07,4.673300e+05,5.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.366019e-01,7.525668e-02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.367237e+08,1.634092e+06,1.990000e+02,6.666667e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.433628e-01,7.525668e-02,...,0.000000e+00,0.000000e+00,4.480431e-03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.260987e+08,6.254382e+06,6.840000e+02,1.000000e+00,9.393939e-01,0.000000e+00,9.230769e-01,0.000000e+00,5.863080e-01,1.505134e-01,...,0.000000e+00,0.000000e+00,7.179541e-02,4.785956e-02,5.936040e-02,4.587383e-02,2.909834e-02,5.172123e-02,6.458231e-02,1.096827e-01
max,3.617734e+08,5.973444e+07,8.297600e+04,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


# Filter columns

In [11]:
from wmeijer_utils.regex import get_matching

pat1 = r'\b.*LinkIntensity.*'
# pat1 = r'\b.*DegreeCentrality.*'
# pat = r'\b(?!=.*Dependency).*Ecosystem.*'

matching = list(get_matching(df.columns, pat1))
print(matching)

df = df[matching]

['ln(1 + EcosystemIntegratorToSubmitterLinkIntensity)', 'ln(1 + EcosystemSubmitterToIntegratorLinkIntensity)', 'ln(1 + IntraProjectIntegratorToSubmitterLinkIntensity)', 'ln(1 + IntraProjectSubmitterToIntegratorLinkIntensity)']


### Predictor Zero Counts

Counts how many fields have partial zero data to see if they are worth including in this study

In [12]:
import regex as re
from typing import Callable


def count_zeroes(_df: pd.DataFrame):
    zeroes = pd.DataFrame()
    zeroes['zeroes'] = _df.eq(0).sum()
    zeroes['non_zeroes'] = len(_df) - zeroes['zeroes']
    zeroes['percentage_zeroes'] = (zeroes['zeroes'] / len(_df)) * 100

    binary_fields = _df.select_dtypes(exclude='number').columns
    ratio_fields = [field for field in _df.columns if len(
        re.findall(r'.*SuccessRate.*', field)) > 0]

    def get_fields_with_requirements(req: Callable[[str, dict], bool]) -> list[str]:
        included_rows = []
        for name, row_data in zeroes.iterrows():
            # We're only interested in continuous data here.
            # The ratio fields have a meaningful 0.
            if name in binary_fields or name in ratio_fields:
                continue

            if req(name, row_data):
                included_rows.append(name)
                print(
                    f'{name}: {row_data["zeroes"]:.0f} zeroes, {row_data["non_zeroes"]:.0f} non-zeroes ({row_data["percentage_zeroes"]:.03f}% zeroes).')

        return included_rows

    rare_cutoff = 80
    print(f"\nRare events ({rare_cutoff}+% zeroes):")
    rare_events = get_fields_with_requirements(lambda _, row_data:
                                               row_data['percentage_zeroes'] > rare_cutoff)

    print("\nNon-rare events:")
    non_rare_events = get_fields_with_requirements(lambda name, _:
                                                   name not in rare_events)

    return rare_events, non_rare_events


### Rare events over all data

In [13]:
rare_events, non_rare_events = count_zeroes(df)

print("\nEvent counts:")
print(f'{len(rare_events)=}')
print(f'{len(non_rare_events)=}')


Rare events (80+% zeroes):

Non-rare events:
ln(1 + EcosystemIntegratorToSubmitterLinkIntensity): 879387 zeroes, 336834 non-zeroes (72.305% zeroes).
ln(1 + EcosystemSubmitterToIntegratorLinkIntensity): 848569 zeroes, 367652 non-zeroes (69.771% zeroes).
ln(1 + IntraProjectIntegratorToSubmitterLinkIntensity): 824271 zeroes, 391950 non-zeroes (67.773% zeroes).
ln(1 + IntraProjectSubmitterToIntegratorLinkIntensity): 679868 zeroes, 536353 non-zeroes (55.900% zeroes).

Event counts:
len(rare_events)=0
len(non_rare_events)=4


### Rare events for any variable.

In [14]:
import regex as re

df.select_dtypes(include='number')
independent_fields = [field for field in df.columns[meta_header_count:]
                      if len(list(re.findall(r'.*Control.*', field))) == 0]
print(independent_fields)
independent_df = df[df[independent_fields].gt(0).any(axis=1)]
print(f'{len(independent_df)=}\n')

rare_events, non_rare_events = count_zeroes(independent_df)

print("\nEvent Counts:")
print(f'{len(rare_events)=}')
print(f'{len(non_rare_events)=}')


[]
len(independent_df)=0


Rare events (80+% zeroes):

Non-rare events:
ln(1 + EcosystemIntegratorToSubmitterLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + EcosystemSubmitterToIntegratorLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + IntraProjectIntegratorToSubmitterLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + IntraProjectSubmitterToIntegratorLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).

Event Counts:
len(rare_events)=0
len(non_rare_events)=4


### Rare events over ecosystem experience

It performs the same test but now on entries that have some form of ecosystem experience.

In [15]:
import regex as re

ecosystem_fields = [field for field in df.columns[meta_header_count:] if len(list(
    re.findall(r'.*Ecosystem.*', field))) > 0]
print(ecosystem_fields)
ecosystem_df = df[df[ecosystem_fields].gt(0).any(axis=1)]
print(f'{len(ecosystem_df)=}\n')

rare_events, non_rare_events = count_zeroes(ecosystem_df)

print("\nEvent Counts:")
print(f'{len(rare_events)=}')
print(f'{len(non_rare_events)=}')

[]
len(ecosystem_df)=0


Rare events (80+% zeroes):

Non-rare events:
ln(1 + EcosystemIntegratorToSubmitterLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + EcosystemSubmitterToIntegratorLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + IntraProjectIntegratorToSubmitterLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + IntraProjectSubmitterToIntegratorLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).

Event Counts:
len(rare_events)=0
len(non_rare_events)=4


### Rare events in dependency ecosystems

In [16]:
import regex as re

dep_ecosystem_fields = [field for field in df.columns[meta_header_count:] if len(list(
    re.findall(r'.*DependencyEcosystem.*', field))) > 0]
print(dep_ecosystem_fields)
dep_ecosystem_df = df[df[dep_ecosystem_fields].gt(0).any(axis=1)]
print(f'{len(dep_ecosystem_df)=}\n')

rare_events, non_rare_events = count_zeroes(dep_ecosystem_df)

print("\nEvent Counts:")
print(f'{len(rare_events)=}')
print(f'{len(non_rare_events)=}')

[]
len(dep_ecosystem_df)=0


Rare events (80+% zeroes):

Non-rare events:
ln(1 + EcosystemIntegratorToSubmitterLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + EcosystemSubmitterToIntegratorLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + IntraProjectIntegratorToSubmitterLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + IntraProjectSubmitterToIntegratorLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).

Event Counts:
len(rare_events)=0
len(non_rare_events)=4


### Rare events in shared experience

In [17]:
import regex as re

shared_experience_fields = [field for field in df.columns[meta_header_count:]
                            if len(list(re.findall(r'.*SharedExperience.*', field))) > 0]
print(shared_experience_fields)
shared_experience_df = df[df[shared_experience_fields].gt(0).any(axis=1)]
print(f'{len(shared_experience_df)=}\n')

rare_events, non_rare_events = count_zeroes(dep_ecosystem_df)

print("\nEvent Counts:")
print(f'{len(rare_events)=}')
print(f'{len(non_rare_events)=}')


[]
len(shared_experience_df)=0


Rare events (80+% zeroes):

Non-rare events:
ln(1 + EcosystemIntegratorToSubmitterLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + EcosystemSubmitterToIntegratorLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + IntraProjectIntegratorToSubmitterLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).
ln(1 + IntraProjectSubmitterToIntegratorLinkIntensity): 0 zeroes, 0 non-zeroes (nan% zeroes).

Event Counts:
len(rare_events)=0
len(non_rare_events)=4


In [18]:
import datetime

print(datetime.datetime.now())

2024-02-16 14:51:37.609150
